### Use Antagonist to train a symptom detection model 

#### Reproducibility

Set seeds to ensure reproducible results.

In [1]:
# Torch
import torch
torch.manual_seed(0)
torch.use_deterministic_algorithms(True)

# Python
import random
random.seed(0)

# Numpy
import numpy as np
np.random.seed(0)

#### Dataset preparation

Note: the dataset needs to be downloaded using the script `download_SMD_dataset.sh` in the `scripts/antagonist-ml` folder.

In [2]:
import os 
from utils import SMD

In [3]:
data_folder = r"..\..\..\data"
db = SMD(dataset_folder=os.path.join(data_folder,'ServerMachineDataset'))

In [4]:
# Reading both training and test data
group = "Group 1"
dataframes_train, _ = db.read_dataset(group_name=group, train=True, retrieve_labels=True)
dataframes, files = db.read_dataset(group_name=group, train=False, retrieve_labels=True)

In [5]:
# Select a subserice given the group (machin-1-1.txt)
service_idx = 0
df,service_id = dataframes[service_idx], files[service_idx][:-4]
df,labels = df[df.columns[:-2].tolist()+['timestamp']], df[['label']]
df_train = dataframes_train[service_idx]
df_train = df_train[df_train.columns[:-1].tolist()+['timestamp']]

In [6]:
# Get interpretation labels (symptoms)
network_incidents = db.get_interpretation_labels(service_id+'.txt')

#### ML Model Loading

In [7]:
import datetime
import pandas as pd
from ml_ad import AENetworkAnomaly

In [8]:
# Filter up to current day to simulate the predition on the next one
current_day = datetime.datetime(2020, 1, 13)
next_day = current_day + datetime.timedelta(days=1)

In [9]:
# Create models folder
models_folder = os.path.join(data_folder,'models')
os.makedirs(models_folder,exist_ok=True)

In [10]:
model_name = f'ae_model_{(current_day).strftime("%Y%m%d")}'
model_folder = os.path.join(models_folder, model_name)

In [11]:
if os.path.exists(model_folder):
    # Load the model if it exist
    ml_model = AENetworkAnomaly.load(model_folder)
else:
    # Create new model 
    ml_model = AENetworkAnomaly(n_inputs=df.shape[1]-1)

    # Get data up to current day (training set)
    df_today = df.loc[df["timestamp"] < current_day.ctime()]

    mask_train = labels[: df_today.shape[0]]["label"] == 0
    df_today = pd.concat([df_train, df_today[mask_train]], ignore_index=True)

    # Train the model
    X_train = df_today.drop('timestamp',axis=1).values
    ml_model.fit(X_train)

    # Cache the trained model
    ml_model.store(model_folder)

#### Use model for detection

Predict anomalies on the current day data.

In [12]:
df_pred = df.loc[
    (df["timestamp"] >= current_day.ctime())
    & (df["timestamp"] < next_day.ctime())
]

X_pred = df_pred.drop('timestamp',axis=1).values
y_pred = ml_model.predict(X_pred, aggregate=False)
model_predictions = ml_model.parse_predictions(df_pred, y_pred)

### Convert to objects

In the following code overlapping symptoms are aggregated into incidents. The code iterates over the sorted list of symptoms and groups them into incidents based on their overlapping time ranges.

In [13]:
# aggregate overlapping symptoms coming from different metrics
day_symptoms = [
    (metric_id, symptom[0], symptom[1]) for metric_id, symptoms_list in model_predictions.items() for symptom in symptoms_list 
]

# sort by starting timestamp
day_symptoms.sort(key=lambda x: x[1])

if len(day_symptoms) > 0:
    # create a list of incident in the form [(start_timestamp, end_timestamp, [symptom1, symptom2]),...]
    start = day_symptoms[0][1] 
    end = day_symptoms[0][2]
    network_incidents = [[start, end, [day_symptoms[0]]]]
    for symptom in day_symptoms[1:]:
        # if overlapping add to the current incident, new incident otherwise
        if symptom[1] <= end:
            network_incidents[-1][2].append(symptom)
            end = max(end, symptom[2])
            network_incidents[-1][1] = end
        else:
            start = symptom[1]
            end = symptom[2]
            network_incidents.append([start, end, [symptom]])

#### Store into antagonist

In [14]:
# Import the antagonist_ml methods
import sys 
sys.path.append("..")
from antagonist_ml.service import store_network_anomalies_labels, store_network_symptom_labels

In [15]:
for network_incident in network_incidents:

    # Create network incident label
    ni_uuid = store_network_anomalies_labels(
        antagonist_host='localhost:5001',
        author_name=model_name,
        author_type="algorithm",
        author_version=1,
        description=f'Detected Network Anomaly on {service_id} - {datetime.datetime.fromtimestamp(network_incident[0]).strftime("%Y-%m-%d at %H")}',
        state="incident-potential",
        version=1,
    )

    # Create network symptoms labels and link with the network incident
    for symptom in network_incident[2]:
        store_network_symptom_labels(    
            antagonist_host='localhost:5001',
            author_name=model_name,
            author_type="algorithm",
            author_version=1,
            confidence=1.0,
            concern_score=0.9,
            description=f"Detected Symptom on {db.get_metric_names()[symptom[0]]} of {service_id}",
            start=symptom[1],
            end=symptom[2],
            version=1,
            tags={"machine": service_id, "metric": db.get_metric_names()[symptom[0]], "group": group},
            network_anomaly_uuid=ni_uuid,
        )